In [1]:
import numpy as np
import pandas as pd

In [2]:
VOCAB_SIZE = 2500

TRAIN_DATA_FILE = 'spam-data/train-data.txt'
TEST_DATA_FILE = 'spam-data/test-data.txt'

# *Load the data generated from the pre_proccess stage*

In [3]:
%%time

sparse_train_data = np.loadtxt(TRAIN_DATA_FILE, delimiter=' ', dtype=int)
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter=' ', dtype=int)

# CPU times: user 4.44 s, sys: 60.3 ms, total: 4.5 s
# Wall time: 4.51 s

CPU times: user 4.44 s, sys: 60.3 ms, total: 4.5 s
Wall time: 4.51 s


In [4]:
# quick check the data loaded
print('train:\n', sparse_train_data[:3], '\n---\n',sparse_train_data[-3:])
print('test: \n', sparse_test_data[:3], '\n---\n', sparse_test_data[-3:])

train:
 [[0 1 1 1]
 [0 5 1 1]
 [0 6 1 1]] 
---
 [[5795 2144    0    1]
 [5795 2267    0    1]
 [5795 2303    0    1]]
test: 
 [[8 0 1 2]
 [8 2 1 1]
 [8 3 1 1]] 
---
 [[5793 1872    0    1]
 [5793 1964    0    1]
 [5793 2310    0    1]]


In [5]:
print('Rows in TRAINING file: ', sparse_train_data.shape[0])
print('Emails in TRAINING file: ', np.unique(sparse_train_data[:, 0]).size)
print('\n')
print('Rows in TESTING file: ', sparse_test_data.shape[0])
print('Emails in TESTING file: ', np.unique(sparse_test_data[:, 0]).size)

Rows in TRAINING file:  264056
Emails in TRAINING file:  4016


Rows in TESTING file:  112088
Emails in TESTING file:  1721


---